In [1]:
import torch
import torchani
import glob
import cclib
import pandas as pd

In [2]:
# path to ANI-2x subdirectories
path = '/home/dakota/Projects/ml-benchmark/ani-2x'

In [3]:
import numpy as np
from math import sqrt
from chemreps.utils.molecule import Molecule
import re

numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts


In [4]:
# for ANI-2x from sae
atom_selfE = {
'H': -0.5975740895940036,
'C': -38.08963824205658,
'N': -54.71039678680539,
'O': -75.19115551439664,
'S': -398.1787966261814,
'F': -99.79839227891235,
'Cl': -460.206968972609
}

atom_conv = {1: 0, 6: 1, 7: 2, 8: 3, 16: 4, 9: 5, 17: 6}
# atom_conv = {0: 1, 1: 6, 2: 7, 3: 6, 4: 16, 5: 9, 6: 17}

device = torch.device('cpu')
const_file = path + '/rHCNOSFCl-5.1R_16-3.5A_a8-4.params'  # noqa: E501
consts = torchani.neurochem.Constants(const_file)
aev_computer = torchani.AEVComputer(**consts)
sae_file = path + '/sae_linfit.dat'  # noqa: E501
energy_shifter = torchani.neurochem.load_sae(sae_file)
model_prefix = path + '/train'  # noqa: E501
ensemble = torchani.neurochem.load_model_ensemble(consts.species, model_prefix, 8)  # noqa: E501

model = torch.nn.Sequential(aev_computer, ensemble, energy_shifter).to(device)

### Bond Stretch

In [5]:
results = []
for out in sorted(glob.iglob('molecules/stretch/*/sdf/*.sdf'), key=numericalSort):
    try:
        name = out.split('ch/')[1].split('/sdf')[0]
        pt = out.split('sdf/')[1].split('.')[0]
        mol = Molecule(out)
        atoms = []
        for i in mol.at_num:
            atoms.append(atom_conv[i])
        coords = mol.xyz.tolist()

        coordinates = torch.tensor([coords],
                               requires_grad=True, device=device)
        species = torch.tensor([atoms], device=device)

        _, energy = model((species, coordinates))
        
        # convert hartree to eV
        e_ev = energy.item() * 27.2114

        d = {}
        d.update({'name': name})
        d.update({'point': pt})
        d.update({'ani2_energy': e_ev})
    #         d.update({'force': list(force.cpu().data.numpy())})
        results.append(d)

        print('Name:{}/{} \tEnergy:{} \tEnergy eV:{}'.format(name, pt, energy.item(), 27.2114*(energy.item())))
    except:
        print('File {} did not run'.format(out))

Name:C6H6-C/1 	Energy:-230.06175974758716 	Energy eV:-6260.302569195494
Name:C6H6-C/2 	Energy:-231.1171939745769 	Energy eV:-6289.0224121198025
Name:C6H6-C/3 	Energy:-231.81451673062892 	Energy eV:-6307.997540563836
Name:C6H6-C/4 	Energy:-232.09666966090353 	Energy eV:-6315.675316810711
Name:C6H6-C/5 	Energy:-232.17877986656876 	Energy eV:-6317.90965046115
Name:C6H6-C/6 	Energy:-232.13602330444755 	Energy eV:-6316.746184546644
Name:C6H6-C/7 	Energy:-232.01329082252832 	Energy eV:-6313.4064618881475
Name:C6H6-C/8 	Energy:-231.8598408773765 	Energy eV:-6309.230874050643
Name:C6H6-C/9 	Energy:-231.65787131341548 	Energy eV:-6303.734999457874
Name:C6H6-C/10 	Energy:-231.39826289566608 	Energy eV:-6296.6706909591285
Name:C6H6-C/11 	Energy:-231.20523879679294 	Energy eV:-6291.4182349950515
Name:C6H6-C/12 	Energy:-231.13100603970142 	Energy eV:-6289.398257748731
Name:C6H6-C/13 	Energy:-231.10178748520465 	Energy eV:-6288.603179974898
Name:C6H6-C/14 	Energy:-231.04778698834033 	Energy eV:-6287

Name:CH3OH/32 	Energy:-115.38080180088232 	Energy eV:-3139.6731501245295
Name:CH3OH/33 	Energy:-115.3818764428253 	Energy eV:-3139.7023926362967
Name:CH3OH/34 	Energy:-115.38217005530547 	Energy eV:-3139.7103822429394
Name:CH3OH/35 	Energy:-115.38231709996413 	Energy eV:-3139.714383533964
Name:CH3OH/36 	Energy:-115.38283473650168 	Energy eV:-3139.728469148842
Name:CH3OH/37 	Energy:-115.38355502883147 	Energy eV:-3139.748069311545
Name:CH3OH/38 	Energy:-115.38410026231955 	Energy eV:-3139.762905878082
Name:CH3OH/39 	Energy:-115.38441473642538 	Energy eV:-3139.771463158766
Name:CH3OH/40 	Energy:-115.38457528153609 	Energy eV:-3139.7758318159913
Name:CH3OH/41 	Energy:-115.38463542262267 	Energy eV:-3139.7774683391544
Name:CH3OH/42 	Energy:-115.38464430371474 	Energy eV:-3139.7777100061035
Name:CH3OH/43 	Energy:-115.38464430371474 	Energy eV:-3139.7777100061035
Name:CH3OH/44 	Energy:-115.38464433351706 	Energy eV:-3139.7777108170662
Name:CH3OH/45 	Energy:-115.38464433351706 	Energy eV:-313

Name:H2/16 	Energy:-1.0746724137630426 	Energy eV:-29.243340919871656
Name:H2/17 	Energy:-1.0499505305256807 	Energy eV:-28.57062386634651
Name:H2/18 	Energy:-1.0176948362555467 	Energy eV:-27.692901267284185
Name:H2/19 	Energy:-0.9868595653738939 	Energy eV:-26.853830377215175
Name:H2/20 	Energy:-0.9593905115094148 	Energy eV:-26.106358964887292
Name:H2/21 	Energy:-0.9348823034729921 	Energy eV:-25.439456312724978
Name:H2/22 	Energy:-0.9119879507985078 	Energy eV:-24.816468924358517
Name:H2/23 	Energy:-0.892701467272278 	Energy eV:-24.291656706532866
Name:H2/24 	Energy:-0.8798197710480653 	Energy eV:-23.941127717897324
Name:H2/25 	Energy:-0.8725610280003511 	Energy eV:-23.743607157328753
Name:H2/26 	Energy:-0.8678708875145875 	Energy eV:-23.61598186851445
Name:H2/27 	Energy:-0.8647723400559388 	Energy eV:-23.531666054198176
Name:H2/28 	Energy:-0.8627415502037965 	Energy eV:-23.476405419215588
Name:H2/29 	Energy:-0.8619576895203553 	Energy eV:-23.455075472614197
Name:H2/30 	Energy:-0.8

Name:HCCH/9 	Energy:-77.12425272912533 	Energy eV:-2098.658890713321
Name:HCCH/10 	Energy:-77.06211736053974 	Energy eV:-2096.968100344591
Name:HCCH/11 	Energy:-77.01308452338726 	Energy eV:-2095.6338481997004
Name:HCCH/12 	Energy:-76.97972922772915 	Energy eV:-2094.726203907429
Name:HCCH/13 	Energy:-76.96317606420071 	Energy eV:-2094.275769153391
Name:HCCH/14 	Energy:-76.94620179385693 	Energy eV:-2093.8138754933584
Name:HCCH/15 	Energy:-76.95284997672589 	Energy eV:-2093.994781856679
Name:HCCH/16 	Energy:-76.96995287389309 	Energy eV:-2094.4601756326547
Name:HCCH/17 	Energy:-76.96682321280987 	Energy eV:-2094.3750131730544
Name:HCCH/18 	Energy:-76.95703404397518 	Energy eV:-2094.1086361842263
Name:HCCH/19 	Energy:-76.95257356018574 	Energy eV:-2093.9872601756383
Name:HCCH/20 	Energy:-76.92998840660603 	Energy eV:-2093.3726865275194
Name:HCCH/21 	Energy:-76.87034373731167 	Energy eV:-2091.749671573483
Name:HCCH/22 	Energy:-76.81939619035275 	Energy eV:-2090.3633174941647
Name:HCCH/23 

Name:N2/20 	Energy:-108.90328907145478 	Energy eV:-2963.410960238985
Name:N2/21 	Energy:-108.92468824161507 	Energy eV:-2963.9932616178844
Name:N2/22 	Energy:-108.9765342093847 	Energy eV:-2965.4040629852507
Name:N2/23 	Energy:-109.04534795059182 	Energy eV:-2967.2765812227344
Name:N2/24 	Energy:-109.12623827828385 	Energy eV:-2969.477720285693
Name:N2/25 	Energy:-109.21377722097851 	Energy eV:-2971.8597774709347
Name:N2/26 	Energy:-109.28597497118928 	Energy eV:-2973.82437933102
Name:N2/27 	Energy:-109.33325951470353 	Energy eV:-2975.1110579584038
Name:N2/28 	Energy:-109.361785925164 	Energy eV:-2975.887301524008
Name:N2/29 	Energy:-109.37521251214959 	Energy eV:-2976.2526577531075
Name:N2/30 	Energy:-109.3777874253533 	Energy eV:-2976.322724746259
Name:N2/31 	Energy:-109.37573996855237 	Energy eV:-2976.267010580266
Name:N2/32 	Energy:-109.37380006460764 	Energy eV:-2976.214223078064
Name:N2/33 	Energy:-109.37295186443545 	Energy eV:-2976.191142363899
Name:N2/34 	Energy:-109.372693545

Name:aspartame/7 	Energy:-1029.72844660363 	Energy eV:-28020.352651910016
Name:aspartame/8 	Energy:-1029.7078508742788 	Energy eV:-28019.792213280354
Name:aspartame/9 	Energy:-1029.6845659025648 	Energy eV:-28019.158596601053
Name:aspartame/10 	Energy:-1029.6618907161692 	Energy eV:-28018.541573033966
Name:aspartame/11 	Energy:-1029.6422967918852 	Energy eV:-28018.008394922705
Name:aspartame/12 	Energy:-1029.6242603816727 	Energy eV:-28017.51759894985
Name:aspartame/13 	Energy:-1029.6085358121136 	Energy eV:-28017.089711397748
Name:aspartame/14 	Energy:-1029.5982472010592 	Energy eV:-28016.8097438869
Name:aspartame/15 	Energy:-1029.5899835833052 	Energy eV:-28016.58487927875
Name:aspartame/16 	Energy:-1029.5832733221987 	Energy eV:-28016.40228367968
Name:aspartame/17 	Energy:-1029.5787052818278 	Energy eV:-28016.27798090593
Name:aspartame/18 	Energy:-1029.576142326806 	Energy eV:-28016.208239311647
Name:aspartame/19 	Energy:-1029.575597197626 	Energy eV:-28016.19340558348
Name:aspartam

Name:gly-gly/6 	Energy:-492.31431564090934 	Energy eV:-13396.561768631042
Name:gly-gly/7 	Energy:-492.29767993984905 	Energy eV:-13396.109087915209
Name:gly-gly/8 	Energy:-492.2765721618959 	Energy eV:-13395.534715726215
Name:gly-gly/9 	Energy:-492.2536220192739 	Energy eV:-13394.91021021527
Name:gly-gly/10 	Energy:-492.2311937153169 	Energy eV:-13394.299904664975
Name:gly-gly/11 	Energy:-492.2115619420835 	Energy eV:-13393.765696630811
Name:gly-gly/12 	Energy:-492.1953790873119 	Energy eV:-13393.32533849648
Name:gly-gly/13 	Energy:-492.18466380013314 	Energy eV:-13393.033760530943
Name:gly-gly/14 	Energy:-492.176551194472 	Energy eV:-13392.813005173255
Name:gly-gly/15 	Energy:-492.1697130590507 	Energy eV:-13392.626929935053
Name:gly-gly/16 	Energy:-492.16494489608493 	Energy eV:-13392.497181545326
Name:gly-gly/17 	Energy:-492.1615583538839 	Energy eV:-13392.405028990877
Name:gly-gly/18 	Energy:-492.1584714479574 	Energy eV:-13392.32102995895
Name:gly-gly/19 	Energy:-492.1563618924298

In [6]:
df = pd.DataFrame(results, columns=['name', 'point', 'ani2_energy'])
df

,name,point,ani2_energy
0,C6H6-C,1,-6260.302569
1,C6H6-C,2,-6289.022412
2,C6H6-C,3,-6307.997541
3,C6H6-C,4,-6315.675317
4,C6H6-C,5,-6317.909650
...,...,...,...
784,gly-gly,41,-13392.256556
785,gly-gly,42,-13392.258657
786,gly-gly,43,-13392.260095
787,gly-gly,44,-13392.260722


In [7]:
df.to_csv('data/ani2-results.csv', index=False)

### Dihedral

In [8]:
results = []
for out in sorted(glob.iglob('molecules/dihedral/*/sdf/*.sdf'), key=numericalSort):
    try:
        name = out.split('ral/')[1].split('/sdf')[0]
        if name == 'sucrose':
            phi = np.nan
            psi = np.nan
            theta = out.split('sdf/')[1].split('.')[0].split('_')[1]
        else:
            phi = out.split('sdf/')[1].split('.')[0].split('_')[1]
            psi = out.split('sdf/')[1].split('.')[0].split('_')[2]
            theta = np.nan
        mol = Molecule(out)
        atoms = []
        for i in mol.at_num:
            atoms.append(atom_conv[i])
        coords = mol.xyz.tolist()

        coordinates = torch.tensor([coords],
                               requires_grad=True, device=device)
        species = torch.tensor([atoms], device=device)

        _, energy = model((species, coordinates))
        
        # convert hartree to eV
        e_ev = energy.item() * 27.2114

        d = {}
        d.update({'name': name})
        d.update({'phi': phi})
        d.update({'psi': psi})
        d.update({'theta': theta})
        d.update({'ani2_energy': e_ev})
    #         d.update({'force': list(force.cpu().data.numpy())})
        results.append(d)

        print('Name:{}/{}/{}/{} \tEnergy:{} \tEnergy eV:{}'.format(name, phi, psi, theta, energy.item(), 27.2114*(energy.item())))
    except:
        print('File {} did not run'.format(out))

Name:ala-ala/0/0/nan 	Energy:-570.8979099461308 	Energy eV:-15534.931386708144
Name:ala-ala/0/20/nan 	Energy:-570.8801149198284 	Energy eV:-15534.44715912942
Name:ala-ala/0/40/nan 	Energy:-570.7470023737719 	Energy eV:-15530.824980393658
Name:ala-ala/0/60/nan 	Energy:-570.9013523080816 	Energy eV:-15535.025058196134
Name:ala-ala/0/80/nan 	Energy:-570.7662464485934 	Energy eV:-15531.348638611255
Name:ala-ala/0/100/nan 	Energy:-570.7261139818897 	Energy eV:-15530.256578006794
Name:ala-ala/0/120/nan 	Energy:-570.9084573158493 	Energy eV:-15535.218395404503
Name:ala-ala/0/140/nan 	Energy:-570.762071452426 	Energy eV:-15531.235031120545
Name:ala-ala/0/160/nan 	Energy:-570.8702095815411 	Energy eV:-15534.177621007148
Name:ala-ala/0/180/nan 	Energy:-570.8965072551241 	Energy eV:-15534.893217522085
Name:ala-ala/0/-20/nan 	Energy:-569.9008692273369 	Energy eV:-15507.800512892756
Name:ala-ala/0/-40/nan 	Energy:-570.9008216926327 	Energy eV:-15535.010619406907
Name:ala-ala/0/-60/nan 	Energy:-570.

Name:ala-ala/100/160/nan 	Energy:-570.8422290602198 	Energy eV:-15533.416231849265
Name:ala-ala/100/180/nan 	Energy:-570.912973753499 	Energy eV:-15535.341293995965
Name:ala-ala/100/-20/nan 	Energy:-570.81052440344 	Energy eV:-15532.553503751767
Name:ala-ala/100/-40/nan 	Energy:-570.9110016295185 	Energy eV:-15535.28762974148
Name:ala-ala/100/-60/nan 	Energy:-570.8326217004528 	Energy eV:-15533.154802139703
Name:ala-ala/100/-80/nan 	Energy:-570.8968938657513 	Energy eV:-15534.903737738505
Name:ala-ala/100/-100/nan 	Energy:-570.9107960232964 	Energy eV:-15535.282034908327
Name:ala-ala/100/-120/nan 	Energy:-570.2743493684998 	Energy eV:-15517.963430405995
Name:ala-ala/100/-140/nan 	Energy:-570.9122358330956 	Energy eV:-15535.321214148697
Name:ala-ala/100/-160/nan 	Energy:-570.8841227957478 	Energy eV:-15534.556219044212
Name:ala-ala/100/-180/nan 	Energy:-570.8444820263853 	Energy eV:-15533.477538212783
Name:ala-ala/120/0/nan 	Energy:-570.9213257500401 	Energy eV:-15535.56856351464
Name:a

Name:ala-ala/-20/-160/nan 	Energy:-570.9053169557324 	Energy eV:-15535.132941809215
Name:ala-ala/-20/-180/nan 	Energy:-570.9154244431725 	Energy eV:-15535.407980692944
Name:ala-ala/-40/0/nan 	Energy:-570.9251970717182 	Energy eV:-15535.673907597355
Name:ala-ala/-40/20/nan 	Energy:-570.9202033647766 	Energy eV:-15535.538021840282
Name:ala-ala/-40/40/nan 	Energy:-570.9197051891556 	Energy eV:-15535.524465784189
Name:ala-ala/-40/60/nan 	Energy:-570.9129817554226 	Energy eV:-15535.341511739507
Name:ala-ala/-40/80/nan 	Energy:-570.9142423787584 	Energy eV:-15535.375815065348
Name:ala-ala/-40/100/nan 	Energy:-570.9271615216007 	Energy eV:-15535.727363028887
Name:ala-ala/-40/120/nan 	Energy:-570.9199289449921 	Energy eV:-15535.530554493758
Name:ala-ala/-40/140/nan 	Energy:-570.9225361266604 	Energy eV:-15535.601499557006
Name:ala-ala/-40/160/nan 	Energy:-570.9144576558342 	Energy eV:-15535.381673055968
Name:ala-ala/-40/180/nan 	Energy:-570.9152323672047 	Energy eV:-15535.402754036953
Name:ala

Name:ala-ala/-140/140/nan 	Energy:-570.9200690457096 	Energy eV:-15535.534366830423
Name:ala-ala/-140/160/nan 	Energy:-570.9142490246763 	Energy eV:-15535.375995910079
Name:ala-ala/-140/180/nan 	Energy:-570.9153922119608 	Energy eV:-15535.407103636551
Name:ala-ala/-140/-20/nan 	Energy:-570.9238496640196 	Energy eV:-15535.637242747503
Name:ala-ala/-140/-40/nan 	Energy:-570.9150021591654 	Energy eV:-15535.396489753914
Name:ala-ala/-140/-60/nan 	Energy:-570.9145387777558 	Energy eV:-15535.383880497024
Name:ala-ala/-140/-80/nan 	Energy:-570.9200524756184 	Energy eV:-15535.533915935042
Name:ala-ala/-140/-100/nan 	Energy:-570.9255348810425 	Energy eV:-15535.683099862
Name:ala-ala/-140/-120/nan 	Energy:-570.9269507297745 	Energy eV:-15535.721627088185
Name:ala-ala/-140/-140/nan 	Energy:-570.9180186906328 	Energy eV:-15535.478573798287
Name:ala-ala/-140/-160/nan 	Energy:-570.9127297469845 	Energy eV:-15535.334654237095
Name:ala-ala/-140/-180/nan 	Energy:-570.918820924448 	Energy eV:-15535.5004

Name:aspartame/40/-80/nan 	Energy:-1029.468968675349 	Energy eV:-28013.291894212394
Name:aspartame/40/-100/nan 	Energy:-1029.4679048218468 	Energy eV:-28013.2629452692
Name:aspartame/40/-120/nan 	Energy:-1029.468654454563 	Energy eV:-28013.283343824896
Name:aspartame/40/-140/nan 	Energy:-1029.4667354159692 	Energy eV:-28013.231124098107
Name:aspartame/40/-160/nan 	Energy:-1029.4655196525791 	Energy eV:-28013.198041474192
Name:aspartame/40/-180/nan 	Energy:-1029.4697404586414 	Energy eV:-28013.312895516276
Name:aspartame/60/0/nan 	Energy:-1029.6290844537953 	Energy eV:-28017.648868706005
Name:aspartame/60/20/nan 	Energy:-1029.6295550771692 	Energy eV:-28017.661675026884
Name:aspartame/60/40/nan 	Energy:-1029.6332187958697 	Energy eV:-28017.761369941927
Name:aspartame/60/60/nan 	Energy:-1029.624540791724 	Energy eV:-28017.52522929992
Name:aspartame/60/80/nan 	Energy:-1029.6289893247822 	Energy eV:-28017.646280112378
Name:aspartame/60/100/nan 	Energy:-1029.6298410006502 	Energy eV:-28017.

...

Name:aspartame/-60/-160/nan 	Energy:-1029.5692334034184 	Energy eV:-28016.020237833778
Name:aspartame/-60/-180/nan 	Energy:-1029.576598272536 	Energy eV:-28016.22064623329
Name:aspartame/-80/0/nan 	Energy:-1029.7147622414568 	Energy eV:-28019.98028125718
Name:aspartame/-80/20/nan 	Energy:-1029.713910222862 	Energy eV:-28019.95709663839
Name:aspartame/-80/40/nan 	Energy:-1029.7144626685122 	Energy eV:-28019.972129457954
Name:aspartame/-80/60/nan 	Energy:-1029.709563762758 	Energy eV:-28019.838823373917
Name:aspartame/-80/80/nan 	Energy:-1029.7120688863733 	Energy eV:-28019.90699129466
Name:aspartame/-80/100/nan 	Energy:-1029.7142103322485 	Energy eV:-28019.96526303495
Name:aspartame/-80/120/nan 	Energy:-1029.7137185045221 	Energy eV:-28019.951879713954
Name:aspartame/-80/140/nan 	Energy:-1029.715658486698 	Energy eV:-28020.004669344933
Name:aspartame/-80/160/nan 	Energy:-1029.7104737170675 	Energy eV:-28019.863584504612
Name:aspartame/-80/180/nan 	Energy:-1029.7124767609575 	Energy eV:-

Name:aspartame/-180/40/nan 	Energy:-1029.4937358953694 	Energy eV:-28013.965844943254
Name:aspartame/-180/60/nan 	Energy:-1029.4896621563175 	Energy eV:-28013.85499280042
Name:aspartame/-180/80/nan 	Energy:-1029.491290286992 	Energy eV:-28013.899296515454
Name:aspartame/-180/100/nan 	Energy:-1029.4933848240116 	Energy eV:-28013.95629180011
Name:aspartame/-180/120/nan 	Energy:-1029.4922923528293 	Energy eV:-28013.92656412978
Name:aspartame/-180/140/nan 	Energy:-1029.4934270241001 	Energy eV:-28013.9574401236
Name:aspartame/-180/160/nan 	Energy:-1029.4912920006254 	Energy eV:-28013.89934314582
Name:aspartame/-180/180/nan 	Energy:-1029.4912510671356 	Energy eV:-28013.898229288257
Name:aspartame/-180/-20/nan 	Energy:-1029.4909520008305 	Energy eV:-28013.8900912754
Name:aspartame/-180/-40/nan 	Energy:-1029.4908978202084 	Energy eV:-28013.88861694482
Name:aspartame/-180/-60/nan 	Energy:-1029.4917333655694 	Energy eV:-28013.911353303858
Name:aspartame/-180/-80/nan 	Energy:-1029.4932824455836 

Name:gly-gly/80/160/nan 	Energy:-492.300899246318 	Energy eV:-13396.19668975126
Name:gly-gly/80/180/nan 	Energy:-492.3011491834947 	Energy eV:-13396.203490891749
Name:gly-gly/80/-20/nan 	Energy:-492.3085705578157 	Energy eV:-13396.405436876947
Name:gly-gly/80/-40/nan 	Energy:-492.30080991385665 	Energy eV:-13396.19425888992
Name:gly-gly/80/-60/nan 	Energy:-492.3014807641336 	Energy eV:-13396.212513665145
Name:gly-gly/80/-80/nan 	Energy:-492.30368987618175 	Energy eV:-13396.272626696733
Name:gly-gly/80/-100/nan 	Energy:-492.30705843268123 	Energy eV:-13396.364289835063
Name:gly-gly/80/-120/nan 	Energy:-492.3092912375757 	Energy eV:-13396.425047582168
Name:gly-gly/80/-140/nan 	Energy:-492.3035086035558 	Energy eV:-13396.2676940148
Name:gly-gly/80/-160/nan 	Energy:-492.29906106887546 	Energy eV:-13396.146670369599
Name:gly-gly/80/-180/nan 	Energy:-492.3043036550113 	Energy eV:-13396.289328477977
Name:gly-gly/100/0/nan 	Energy:-492.2529884666034 	Energy eV:-13394.892970360133
Name:gly-gly/

Name:gly-gly/-20/80/nan 	Energy:-492.307424390299 	Energy eV:-13396.374248054182
Name:gly-gly/-20/100/nan 	Energy:-492.30313446520057 	Energy eV:-13396.25751318636
Name:gly-gly/-20/120/nan 	Energy:-492.30644606946197 	Energy eV:-13396.347626574558
Name:gly-gly/-20/140/nan 	Energy:-492.3023882746526 	Energy eV:-13396.237208296883
Name:gly-gly/-20/160/nan 	Energy:-492.29263926444736 	Energy eV:-13395.971924080583
Name:gly-gly/-20/180/nan 	Energy:-492.30043030677524 	Energy eV:-13396.183929249784
Name:gly-gly/-20/-20/nan 	Energy:-492.30529257057395 	Energy eV:-13396.316238254916
Name:gly-gly/-20/-40/nan 	Energy:-492.2992209881374 	Energy eV:-13396.151021996602
Name:gly-gly/-20/-60/nan 	Energy:-492.29339211581436 	Energy eV:-13395.992410220271
Name:gly-gly/-20/-80/nan 	Energy:-492.3023295342752 	Energy eV:-13396.235609888978
Name:gly-gly/-20/-100/nan 	Energy:-492.3085047393867 	Energy eV:-13396.403645865348
Name:gly-gly/-20/-120/nan 	Energy:-492.3014278799125 	Energy eV:-13396.211074611452

Name:gly-gly/-120/-180/nan 	Energy:-492.2617098331281 	Energy eV:-13395.130290953182
Name:gly-gly/-140/0/nan 	Energy:-492.31142484544006 	Energy eV:-13396.483106039208
Name:gly-gly/-140/20/nan 	Energy:-492.3021722972223 	Energy eV:-13396.231331248635
Name:gly-gly/-140/40/nan 	Energy:-492.30421548484054 	Energy eV:-13396.28692924419
Name:gly-gly/-140/60/nan 	Energy:-492.2994605094024 	Energy eV:-13396.157539705553
Name:gly-gly/-140/80/nan 	Energy:-492.3034149646589 	Energy eV:-13396.265145969319
Name:gly-gly/-140/100/nan 	Energy:-492.3096673130819 	Energy eV:-13396.435281123197
Name:gly-gly/-140/120/nan 	Energy:-492.3049395471641 	Energy eV:-13396.306631993702
Name:gly-gly/-140/140/nan 	Energy:-492.30421658752647 	Energy eV:-13396.28695924982
Name:gly-gly/-140/160/nan 	Energy:-492.3012342095205 	Energy eV:-13396.205804568946
Name:gly-gly/-140/180/nan 	Energy:-492.3010008573362 	Energy eV:-13396.199454729318
Name:gly-gly/-140/-20/nan 	Energy:-492.3082318693229 	Energy eV:-13396.396220688

In [9]:
df = pd.DataFrame(results, columns=['name', 'phi', 'psi', 'theta', 'ani2_energy'])
df

,name,phi,psi,theta,ani2_energy
0,ala-ala,0,0,NaN,-15534.931387
1,ala-ala,0,20,NaN,-15534.447159
2,ala-ala,0,40,NaN,-15530.824980
3,ala-ala,0,60,NaN,-15535.025058
4,ala-ala,0,80,NaN,-15531.348639
...,...,...,...,...,...
1097,sucrose,NaN,NaN,-100,-35276.792379
1098,sucrose,NaN,NaN,-120,-35253.356182
1099,sucrose,NaN,NaN,-140,-35232.691608
1100,sucrose,NaN,NaN,-160,-35277.464470


In [10]:
df.to_csv('data/ani2-dihedral-results.csv', index=False)